
<br>
====================================<br>
Outlier detection on a real data set<br>
====================================<br>
This example illustrates the need for robust covariance estimation<br>
on a real data set. It is useful both for outlier detection and for<br>
a better understanding of the data structure.<br>
We selected two sets of two variables from the Boston housing data set<br>
as an illustration of what kind of analysis can be done with several<br>
outlier detection tools. For the purpose of visualization, we are working<br>
with two-dimensional examples, but one should be aware that things are<br>
not so trivial in high-dimension, as it will be pointed out.<br>
In both examples below, the main result is that the empirical covariance<br>
estimate, as a non-robust one, is highly influenced by the heterogeneous<br>
structure of the observations. Although the robust covariance estimate is<br>
able to focus on the main mode of the data distribution, it sticks to the<br>
assumption that the data should be Gaussian distributed, yielding some biased<br>
estimation of the data structure, but yet accurate to some extent.<br>
The One-Class SVM does not assume any parametric form of the data distribution<br>
and can therefore model the complex shape of the data much better.<br>
First example<br>
-------------<br>
The first example illustrates how robust covariance estimation can help<br>
concentrating on a relevant cluster when another one exists. Here, many<br>
observations are confounded into one and break down the empirical covariance<br>
estimation.<br>
Of course, some screening tools would have pointed out the presence of two<br>
clusters (Support Vector Machines, Gaussian Mixture Models, univariate<br>
outlier detection, ...). But had it been a high-dimensional example, none<br>
of these could be applied that easily.<br>
Second example<br>
--------------<br>
The second example shows the ability of the Minimum Covariance Determinant<br>
robust estimator of covariance to concentrate on the main mode of the data<br>
distribution: the location seems to be well estimated, although the covariance<br>
is hard to estimate due to the banana-shaped distribution. Anyway, we can<br>
get rid of some outlying observations.<br>
The One-Class SVM is able to capture the real data structure, but the<br>
difficulty is to adjust its kernel bandwidth parameter so as to obtain<br>
a good compromise between the shape of the data scatter matrix and the<br>
risk of over-fitting the data.<br>


In [ ]:
print(__doc__)

Author: Virgile Fritsch <virgile.fritsch@inria.fr><br>
License: BSD 3 clause

In [ ]:
import numpy as np
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
import matplotlib.pyplot as plt
import matplotlib.font_manager
from sklearn.datasets import load_boston

Get data

In [ ]:
X1 = load_boston()['data'][:, [8, 10]]  # two clusters
X2 = load_boston()['data'][:, [5, 12]]  # "banana"-shaped

Define "classifiers" to be used

In [ ]:
classifiers = {
    "Empirical Covariance": EllipticEnvelope(support_fraction=1.,
                                             contamination=0.261),
    "Robust Covariance (Minimum Covariance Determinant)":
    EllipticEnvelope(contamination=0.261),
    "OCSVM": OneClassSVM(nu=0.261, gamma=0.05)}
colors = ['m', 'g', 'b']
legend1 = {}
legend2 = {}

Learn a frontier for outlier detection with several classifiers

In [ ]:
xx1, yy1 = np.meshgrid(np.linspace(-8, 28, 500), np.linspace(3, 40, 500))
xx2, yy2 = np.meshgrid(np.linspace(3, 10, 500), np.linspace(-5, 45, 500))
for i, (clf_name, clf) in enumerate(classifiers.items()):
    plt.figure(1)
    clf.fit(X1)
    Z1 = clf.decision_function(np.c_[xx1.ravel(), yy1.ravel()])
    Z1 = Z1.reshape(xx1.shape)
    legend1[clf_name] = plt.contour(
        xx1, yy1, Z1, levels=[0], linewidths=2, colors=colors[i])
    plt.figure(2)
    clf.fit(X2)
    Z2 = clf.decision_function(np.c_[xx2.ravel(), yy2.ravel()])
    Z2 = Z2.reshape(xx2.shape)
    legend2[clf_name] = plt.contour(
        xx2, yy2, Z2, levels=[0], linewidths=2, colors=colors[i])

In [ ]:
legend1_values_list = list(legend1.values())
legend1_keys_list = list(legend1.keys())

Plot the results (= shape of the data points cloud)

In [ ]:
plt.figure(1)  # two clusters
plt.title("Outlier detection on a real data set (boston housing)")
plt.scatter(X1[:, 0], X1[:, 1], color='black')
bbox_args = dict(boxstyle="round", fc="0.8")
arrow_args = dict(arrowstyle="->")
plt.annotate("several confounded points", xy=(24, 19),
             xycoords="data", textcoords="data",
             xytext=(13, 10), bbox=bbox_args, arrowprops=arrow_args)
plt.xlim((xx1.min(), xx1.max()))
plt.ylim((yy1.min(), yy1.max()))
plt.legend((legend1_values_list[0].collections[0],
            legend1_values_list[1].collections[0],
            legend1_values_list[2].collections[0]),
           (legend1_keys_list[0], legend1_keys_list[1], legend1_keys_list[2]),
           loc="upper center",
           prop=matplotlib.font_manager.FontProperties(size=12))
plt.ylabel("accessibility to radial highways")
plt.xlabel("pupil-teacher ratio by town")

In [ ]:
legend2_values_list = list(legend2.values())
legend2_keys_list = list(legend2.keys())

In [ ]:
plt.figure(2)  # "banana" shape
plt.title("Outlier detection on a real data set (boston housing)")
plt.scatter(X2[:, 0], X2[:, 1], color='black')
plt.xlim((xx2.min(), xx2.max()))
plt.ylim((yy2.min(), yy2.max()))
plt.legend((legend2_values_list[0].collections[0],
            legend2_values_list[1].collections[0],
            legend2_values_list[2].collections[0]),
           (legend2_keys_list[0], legend2_keys_list[1], legend2_keys_list[2]),
           loc="upper center",
           prop=matplotlib.font_manager.FontProperties(size=12))
plt.ylabel("% lower status of the population")
plt.xlabel("average number of rooms per dwelling")

In [ ]:
plt.show()